In [ ]:
import pandas as pd
import jieba

In [ ]:
from google.colab import drive

drive.mount('/content/drive')  ## https://blog.csdn.net/sdaujz/article/details/106531632
                 ## https://blog.csdn.net/sdaujz/article/details/106531632


Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Capstone_Project/data

/content/drive/MyDrive/Capstone_Project/data


In [ ]:
data = pd.read_csv("0827_Cecilia.csv")
data.head(3)

,Unnamed: 0,Unnamed: 0.1,URL,Stock_ID,Stock_Name,Publish Time,Title,Article,doc_Jieba,Stock_ID_1,date,close_price,tech_mfi,MACD,BIAS,KDJ,MA,volume,amt,pct_chg,swing,open
0,0,0,http://finance.sina.com.cn/stock/s/2018-11-20/...,['stock_sh600783'],['鲁信创投'],2018年11月20日 16:50,鲁信创投提示风险：最新动态市盈率已为1244倍,"[' 新浪财经讯11月20日消息,鲁信创投提示风险:公司对应2017年每股收益的市盈率为4...","[""'"", '新浪', '财经', '讯', '11', '月', '20', '日', '...",600783.SH,2018-11-20,23.44,72.1422524509452,3.1928700962990195,0,0,0,95125249.0,2.236565e+09,1.868753,9.213385,23.01
1,1,1,https://finance.sina.com.cn/stock/s/2021-07-26...,['stock_sz002496'],['ST辉丰'],2021年07月26日 17:32,辉丰股份索赔案已有胜诉判决 受损投资者可继续加入索赔,[' 受损股民可至新浪股民维权平台登记该公司维权:http://wq.finance.si...,"[""'"", '受损', '股民', '可至', '新浪', '股民', '维权', '平台'...",002496.SZ,2021-07-26,2.82,56.32324980310697,-0.044984413032010284,0,0,0,18595408.0,5.217624e+07,4.832714,6.319703,2.69
2,2,2,https://finance.sina.com.cn/money/fund/jjzl/20...,['stock_sz300814'],['中富电路'],2021年08月11日 22:09,视频直播 | 中富电路8月12日深交所上市仪式,"['中富电路新股发行上市仪式', '08月12日09:00-09:30', '中富电路新股发...","[""'"", '中富', '电路', '新股', '发行', '上市', '仪式', ""'"",...",300814.SZ,2021-08-11,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN


In [ ]:
data['Article'][0]

"['  新浪财经讯11月20日消息,鲁信创投提示风险:公司对应2017年每股收益的市盈率为404倍,最新动态市盈率为1244倍,远高于金融业行业最新动态市盈率,请广大投资者注意投资风险。']"

In [ ]:
# jieba分词的demo【可跳过直接下一步】
import jieba
# 载入自定义词库（比如：所有股票名称/代码
# jieba.load_userdict("./dict/jieba_dict.txt")

jieba.enable_paddle()# 启动paddle模式。 0.40版之后开始支持，早期版本不支持

seg_list = jieba.cut("我来到北京清华大学",use_paddle=True) # 使用paddle模式
print("Paddle Mode: " + '/'.join(list(seg_list)))

seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
print(", ".join(seg_list))

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
print(", ".join(seg_list))

Paddle enabled successfully......
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


Paddle Mode: 我/来到/北京清华大学


Loading model cost 0.838 seconds.
Prefix dict has been built successfully.


Full Mode: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学
Default Mode: 我/ 来到/ 北京/ 清华大学
他, 来到, 了, 网易, 杭研, 大厦
小明, 硕士, 毕业, 于, 中国, 科学, 学院, 科学院, 中国科学院, 计算, 计算所, ，, 后, 在, 日本, 京都, 大学, 日本京都大学, 深造


In [ ]:
# 缺失值处理
# 缺失值查找与填充：https://blog.csdn.net/qq_28811329/article/details/82821971?utm_medium=distribute.pc_relevant_t0.none-task-blog-2%7Edefault%7EBlogCommendFromMachineLearnPai2%7Edefault-1.control&depth_1-utm_source=distribute.pc_relevant_t0.none-task-blog-2%7Edefault%7EBlogCommendFromMachineLearnPai2%7Edefault-1.control
import math 
# 删除含有缺失值的行：https://blog.csdn.net/Guo_ya_nan/article/details/81042882 或者 https://www.delftstack.com/zh/howto/python-pandas/pandas-drop-rows-with-nan/#:~:text=Pandas%20%E4%BD%BF%E7%94%A8%20DataFrame.dropna%20%28%29%20%E6%96%B9%E6%B3%95%E5%8F%AA%E5%88%A0%E9%99%A4%E6%89%80%E6%9C%89%E5%88%97%E9%83%BD%E6%98%AF%20NaN%20%E5%80%BC%E7%9A%84%E8%A1%8C.%20%E5%AE%83%E5%8F%AA%E5%88%A0%E9%99%A4,%E6%88%91%E4%BB%AC%E5%9C%A8%20dropna%20%28%29%20%E6%96%B9%E6%B3%95%E4%B8%AD%E8%AE%BE%E7%BD%AE%20how%3D%27all%27%20%EF%BC%8C%E8%AE%A9%E8%AF%A5%E6%96%B9%E6%B3%95%E5%8F%AA%E5%9C%A8%E8%A1%8C%E7%9A%84%E6%89%80%E6%9C%89%E5%88%97%E5%80%BC%E9%83%BD%E6%98%AF%20NaN%20%E6%97%B6%E6%89%8D%E5%88%A0%E9%99%A4%E8%A1%8C%E3%80%82.
data_copy = data.copy()
data_copy.dropna(axis=0, how='any', inplace=True)
print("删除前：",len(data))
print("删除后：",len(data_copy))
# 删除目标值Y（pct_chg）缺失的行
def is_not_nan(x):
  return not math.isnan(x)
data_copy = data_copy[data_copy['pct_chg'].map(is_not_nan)]
print("删除前：",len(data))
print("删除后：",len(data_copy))

data = data_copy.reset_index()


删除前： 7239
删除后： 5885
删除前： 7239
删除后： 5885


In [ ]:
#删除固定banner位
for i in range(0,len(data)):
    if '金麒麟' in data['Article'][i]:
        data['Article'][i] = data['Article'][i][40:9999999]

#删除Y（pct_chg）缺失的行


def getJiebaList(x):
  # 对字符串'['string1','string2']'结巴分词¶，得到list：[[string1 word],[string2 word]]
  result = []
  y = x.strip("[").strip("]").split(", ")
  for string in y:
    tmp = " ".join(jieba.cut(string.strip("\'").strip('\\u3000\\u3000').strip('\u3000\u3000'))).split()
    result.append(tmp)
  return result

def transListsToList(x):
  # 将包含多个lists的list x合并为1个list(即一篇文章的词汇存在1个list中)
  return [i for p in x for i in p]


data['doc_Jieba_List']=data.Article.apply(getJiebaList)
data['doc_Jieba']=data.doc_Jieba_List.apply(transListsToList)
data.head(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.025 seconds.
Prefix dict has been built successfully.


,index,Unnamed: 0,Unnamed: 0.1,URL,Stock_ID,Stock_Name,Publish Time,Title,Article,doc_Jieba,Stock_ID_1,date,close_price,tech_mfi,MACD,BIAS,KDJ,MA,volume,amt,pct_chg,swing,open,doc_Jieba_List
0,0,0,0,http://finance.sina.com.cn/stock/s/2018-11-20/...,['stock_sh600783'],['鲁信创投'],2018年11月20日 16:50,鲁信创投提示风险：最新动态市盈率已为1244倍,"[' 新浪财经讯11月20日消息,鲁信创投提示风险:公司对应2017年每股收益的市盈率为4...","[新浪, 财经, 讯, 11, 月, 20, 日, 消息, ,, 鲁信, 创投, 提示, 风...",600783.SH,2018-11-20,23.44,72.1422524509452,3.1928700962990195,0,0,0,95125249.0,2.236565e+09,1.868753,9.213385,23.01,"[[新浪, 财经, 讯, 11, 月, 20, 日, 消息, ,, 鲁信, 创投, 提示, ..."


In [ ]:
for i in range(len(data['Article'])):
  if '投资收益' in data['Article'][i]:
    print(data['doc_Jieba'][i],'\n')
    print(data['URL'][i])
    break
  

['原', '标题', ':', '云南', '城投', '上半年', '营收', '跌超', '四成', '靠', '卖', '资产', '扭亏为盈', '每经', '记者', '陈利', '每经', '编辑', '魏', '文艺', '变卖', '资产', '成为', '云南', '城投', '(', '*', 'ST', '云城', ':', '600239', ',', 'SH', ';', '昨日', '收盘价', '1.97', '元', ')', '“', '回血', '”', '的', '一贯', '操作', '。', '在', '连续', '的', '“', '卖卖', '卖', '”', '后', ',', '云南', '城投', '回笼资金', '也', '有', '了', '实质性', '进展', '。', '据', '云南', '城投', '近日', '发布', '的', '2021', '年', '半年度', '业绩', '快报', ',', '今年', '上半年', ',', '实现', '营业', '收入', '16.97', '亿元', ',', '同比', '下降', '40.07%', ';', '归属于', '上市公司', '股东', '净利润', '3.05', '亿元', ',', '而', '去年同期', '为', '-', '7.6', '亿元', '。', '持续', '“', '卖卖', '卖', '”', '对于', '已', '“', '戴帽', '”', '的', '云南', '城投', '来说', ',', '首当其冲', '仍', '是', '“', '活下去', '”', '。', '“', '卖卖', '卖', '”', '成为', '云南', '城投', '“', '回血', '”', '的', '常规', '操作', '。', '早', '在', '今年', '4', '月', '9', '日', ',', '云南', '城投', '公告', '宣布', ',', '拟以', '公开', '挂牌', '及', '非', '公开', '协议', '转让', '的', '方式', '对外', '出售', '城', '投置', '地', '、', '安盛', '创享', '、', '民生', '喜神',

In [ ]:
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file) as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list

stop_words_file = './stopwords-master/哈工大停用词表.txt'
stopwords_1 = get_custom_stopwords(stop_words_file)
stop_words_file = './stopwords-master/中文停用词表.txt'
stopwords_2 = get_custom_stopwords(stop_words_file)
stop_words_file = './stopwords-master/四川大学机器智能实验室停用词库.txt'
stopwords_3 = get_custom_stopwords(stop_words_file)
stop_words_file = './stopwords-master/百度停用词表.txt'
stopwords_4 = get_custom_stopwords(stop_words_file)
stopwords=stopwords_1+stopwords_2+stopwords_3+stopwords_4
stopwords=list(set(stopwords))
#去重后2318个停用词

def clean_stopword(x):
    y=[]
    for i in x:
        if i not in stopwords:
            y.append(i)
    return y

data['doc_Jieba_2']=data.doc_Jieba.apply(clean_stopword)

In [ ]:
# Selecting frequently-used words：选择出现次数大于kappa的词汇（kappa为超参数），对应论文中D__freq
## 计算包含词汇i的文章数，存在字典dict_word_ariticles中；然后计算每篇文章的词汇set set_word_ariticles，
dict_word_ariticles = {}
for article_list in data['doc_Jieba_2']:
  set_word_ariticles = set(article_list)
  for word in list(set_word_ariticles):
    if word in dict_word_ariticles:
      dict_word_ariticles[word] += 1
    else:
      dict_word_ariticles[word] = 1
# dict_word_ariticles


## 选择出现次数大于kappa的词汇（kappa为超参数）
kappa = 20
frequent_dict = [k for k, v in dict_word_ariticles.items() if v > kappa]

# 词到ID的映射, 使得每个词有一个ID
D__freq = dict(zip(frequent_dict, range(len(frequent_dict))))
# sorted(D__freq)
len(D__freq)

8423

In [ ]:
tmp_num = 0
for i in range(0,len(data)):
    if '收益' in data['Article'][i]:
        tmp_num += 1
print(tmp_num)
tmp_num = 0
for i in range(0,len(data)):
    if '收益' in data['doc_Jieba_2'][i]:
        tmp_num += 1
print(tmp_num)
print(dict_word_ariticles['收益'] + dict_word_ariticles['收益率'] + dict_word_ariticles['投资收益'] + dict_word_ariticles['收益分配'] + dict_word_ariticles['经济收益'] + dict_word_ariticles['收益权'] + dict_word_ariticles['低收益'])
for k,v in dict_word_ariticles.items():
  if '收益' in k:
    print(k,'\n')
dict_word_ariticles['收益']

1479
1291
2275
收益 

收益率 

投资收益 

收益分配 

经济收益 

收益权 

低收益 



1291

### Factor modeling【todo】
$ X = FB^T + U$

### Learning factors and idiosyncratic components【todo】
（0）求X：

（1.1）指定k，求F：$F=sqrt(n)*(XX^T的最大的前k个特征值）$

（1.2）求出k，再求F

（2） 根据X和F，求B：$B=X^TF/n$

（3）根据X，F和B，求U：$U=X-FB^T$

In [ ]:
# sorted(D__freq.items(), key=lambda item:item[1])

In [ ]:
# （0）求X
from collections import Counter
from itertools import chain
import numpy as np
documents = data['doc_Jieba_2']
def word_matrix(docs, dictionary, word_num, doc_num ):
    '''计算词频矩阵：行表示文档，列表示词语'''
    # 词到ID的映射, 使得每个词有一个ID
    # dictionary = dict(zip(words, range(len(words))))
    #print(dictionary)
    # 创建一个空的矩阵, 行数等于文档数, 列数等于词数
    matrix = np.zeros((doc_num,  word_num))
    # 逐个文档统计词频
    for row in range(doc_num):  # row 表示矩阵第几行，即第几个文档
        # 统计词频
        count = Counter(docs[row])#其实是个词典，词典元素为：{单词：次数}。
        for word in count:
          if word in D__freq:
            # 用word的id表示word在矩阵中的列数，该文档表示行数。
            col= dictionary[word] # col表示矩阵第几列，即第几个单词
            # 把词频赋值给矩阵
            matrix[row,col] = count[word]
    return matrix
 
X = word_matrix(documents, D__freq, len(D__freq), len(documents))
print("Word frequency matrix is:",X) ## [doc_num, word_num]

Word frequency matrix is: [[1. 2. 2. ... 0. 0. 0.]
 [7. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [6. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [ ]:
# （1.1）指定k，求F：$F=sqrt(n)*(XX^T的最大的前k个特征值）$。【todo】检查下算得对不对 ；【todo】这里k是超参数
# 【参考：https://blog.csdn.net/w450468524/article/details/54885773
import numpy as np
n = len(documents)
k = 10

eigenvalue, featurevector = np.linalg.eig(np.matmul(X,X.T))
sorted_indices = np.argsort(eigenvalue)
# 上面得到的sorted_indices就是特征值排序后的结果，巧妙的是这里是用数组下标来表示的，也就是说其中存放的是特征值由小到大的顺序排序时对应的下标[0, 4, 1, 3, 2]，而不是直接存放特征值。
# 下一步就是取前k大的特征向量了：
topk_evecs = featurevector[:,sorted_indices[:-k-1:-1]]

F = np.sqrt(n) * topk_evecs  ## [doc_num, k]
print("F = ",F)

# 复数：
# 实数部分和虚数部分都是浮点数
# 虚数部分必须有j或J
# 下面是些得数：
# 64.23+1j            4.34-8.5j            0.23-8.33j            1.23e-0.45+6.5e+0.83j              -1.23-3.5j            -0.34-0j
# 复数中的内建属性

# （1.2）求出k，再求F：

# （2） 根据X和F，求B：$B=X^TF/n$
B = np.matmul(X.T,F) / n
print("B =",B)  ## [word_num, k]
# （3）根据X，F和B，求U：$U=X-FB^T$
U = X - np.matmul(F,B.T)
print("U =",U) ## [doc_num, word_num]

F =  [[ 7.44923497e-02+0.j -4.74976451e-02+0.j -4.59551534e-02+0.j ...
   7.82556090e-02+0.j  1.63910361e-03+0.j -9.10284751e-03+0.j]
 [ 1.02587115e+00+0.j -5.34284139e-01+0.j -7.57413329e-01+0.j ...
   9.11532618e-01+0.j  3.19272216e-01+0.j -3.32356378e-01+0.j]
 [ 2.78399422e-01+0.j -8.73531063e-02+0.j -4.76542257e-02+0.j ...
   2.79974362e-01+0.j  2.84119983e-02+0.j -7.28093509e-02+0.j]
 ...
 [ 1.35936911e-01+0.j -7.85675197e-02+0.j -3.70571841e-02+0.j ...
   1.86637862e-01+0.j -1.38641848e-01+0.j  4.32531860e-02+0.j]
 [ 6.23156409e-01+0.j -4.98786043e-01+0.j -5.41208829e-01+0.j ...
   1.02220210e+00+0.j -3.38941205e-01+0.j -4.67540973e-01+0.j]
 [ 2.15161334e+00+0.j  2.70182514e+00+0.j  6.71676352e-01+0.j ...
  -1.05830289e-01+0.j -1.93584522e-01+0.j -9.29069900e-02+0.j]]
B = [[ 2.75601483e+00+0.j -1.58801443e+00+0.j -1.10495044e+00+0.j ...
   1.06971623e+00+0.j  6.36665994e-03+0.j  1.46143395e-01+0.j]
 [ 2.82895711e-02+0.j -2.17553326e-02+0.j -1.27529698e-02+0.j ...
   1.22072574e-0

In [ ]:
word_num , doc_num = len(D__freq), len(documents)
assert X.shape == (doc_num, word_num)
assert F.shape == (doc_num, k)
assert B.shape == (word_num, k)
assert U.shape == (doc_num, word_num)


### Learning conditional sentiment-charged words S【todo】

In [ ]:
## 对于Y缺失值，先用平均数填充，之后确定了Y，要在一开始就把包含缺失Y值的data的整行数据删除
data.open = data.open.fillna(data.open.mean())

In [ ]:
data.open

0        23.01
1         2.69
2       276.52
3        39.31
4        91.00
         ...  
5880     15.22
5881     51.05
5882     14.00
5883      9.57
5884      6.15
Name: open, Length: 5885, dtype: float64

In [ ]:
# 1.根据Y和F做线性回归，求残差Y_u
# 先用收盘价（open）作为Y
from numpy import random
Y = data.open
# 判断F的每个复数元素是否为实数：
assert np.all(np.imag(F)==0) #np.imag(F)：提取复数的虚部；np.real(F)：提取复数的实部 
                      #np.any()，np.all()，来判断一个矩阵中是否有0元素或一个矩阵是否是零矩阵
F = np.real(F)
# 1.1.根据Y和F做线性回归，求Y_f_pred
#【参考：https://www.cnblogs.com/learnbydoing/p/12190168.html
from sklearn.linear_model import LinearRegression
model_Y_f = LinearRegression()
model_Y_f = model_Y_f.fit(F, Y) #np.real(F)：提取复数的实部
'''
# get result: y = b0 + b1x
r_sq = model_Y_f.score(F, Y)
print('coefficient of determination(𝑅²) :', r_sq)
# coefficient of determination(𝑅²) : 0.715875613747954
print('intercept:', model_Y_f.intercept_)
# （标量） 系数b0 intercept: 5.633333333333329 -------this will be an array when y is also 2-dimensional
print('slope:', model_Y_f.coef_)
# （数组）斜率b1 slope: [0.54]        ---------this will be 2-d array when y is also 2-dimensional
'''
# predict response: given x, get y from the model y = b0+b1x
Y_f_pred = model_Y_f.predict(F)
# 1.2.求残差Y_u
Y_u = Y - Y_f_pred
Y_u.shape

(5885,)

In [ ]:
# 2.求Y_u与U的相关系数，然后按照论文，根据$S = {j:|corr(U_j,Y_u)|>alpha & {j:k_j>=kappa}$ ，这里alpha和kappa是超参数
#   需要注意的是：因为U_j来自于D_freq，所以已经满足{j:k_j>=kappa}了。 
#   另外，接下来我们会直接取相关度绝对值排前alpha的U_j，所以下面出现的alpha和上述提到论文里含义稍有区别，不过作用是一样的。

# 【希腊字母与英文对照：https://blog.csdn.net/bxlover007/article/details/105725593
#【求相关系数：https://www.cnblogs.com/marszhw/p/12175454.html
import random
import numpy as np
# 判断U的每个复数元素是否为实数：
assert np.all(np.imag(U)==0) #np.imag(U)：提取复数的虚部；np.real(U)：提取复数的实部 
                      #np.any()，np.all()，来判断一个矩阵中是否有0元素或一个矩阵是否是零矩阵
U = np.real(U)
corr_U_j__Y_u = np.zeros(len(D__freq))
for j in range(len(D__freq)):
  tmp = np.array([Y_u, U[:,j]])
  corr_U_j__Y_u[j] = np.corrcoef(tmp)[0][1] # np.cov(tmp) # 计算协方差矩阵 ; np.corrcoef(tmp) # 计算相关系数矩阵
corr_abs_U_j__Y_u = abs(corr_U_j__Y_u)
# 下一步就是取前alpha大的相关度绝对值对应的U_j的下标：
sorted_indices_U = np.argsort(corr_abs_U_j__Y_u)
sorted_indices_U #上面得到的sorted_indices_U就是相关度绝对值排序后的结果，巧妙的是这里是用数组下标来表示的，也就是说其中存放的是相关度绝对值由小到大的顺序排序时对应的下标[0, 4, 1, 3, 2]，而不是直接存放相关度绝对值。
# print("Is min equals:",min(corr_abs_U_j__Y_u)==corr_abs_U_j__Y_u[sorted_indices_U[0]]) # 验证排序正确
alpha = 1000
topk_U_j = sorted_indices_U[:-alpha-1:-1]
# 最后得到情感词汇集S
S = [k for k,v in D__freq.items() if v in topk_U_j]

In [ ]:
print(S)

NameError: ignored

In [ ]:
assert len(S) == 1000
len(D__freq)

8423

In [ ]:
# 生成U_S（对应论文中U_i,S)
U_S = U[:,topk_U_j]

In [ ]:
# 保存数据：
np.save("F.npy",F)
np.save("U_S.npy",U_S)
np.save("Y.npy",np.array(data['pct_chg']))



In [ ]:
# 读取数据
import numpy as np

F = np.load("F.npy")
U_S = np.load("U_S.npy")
Y = np.load("Y.npy")
print("总共有 ",len(Y)," 条数据")

总共有  5885  条数据


In [ ]:
word_num , doc_num = len(D__freq), len(documents)
S_word_num = len(S)
assert X.shape == (doc_num, word_num)
assert F.shape == (doc_num, k)
assert B.shape == (word_num, k)
assert U.shape == (doc_num, word_num)
assert U_S.shape == (doc_num, S_word_num)

In [ ]:
# 模型训练
# tf2实现线性回归：https://blog.csdn.net/Mrwxxxx/article/details/108133137
import tensorflow as tf
import matplotlib.pyplot as plt


# 样本准备
# 划分训练集和测试集
from sklearn.model_selection import train_test_split
test_size = 0.1
random_state = 7600
F_train, F_test, U_S_train, U_S_test, Y_train, Y_test= train_test_split(F, U_S, np.array(data['pct_chg']), test_size=test_size, random_state=random_state)
# U_test.shape: (589, 8423)

#参数
a = tf.Variable(tf.zeros([1], name='a'))
b = tf.Variable(tf.random.uniform([1,k], -1.0, 1.0), name='b')
BETA = tf.Variable(tf.random.uniform([1,S_word_num], -1.0, 1.0), name='BETA')

#需要更新的变量
variables = [a, b, BETA]

#采用梯度下降法优化参数
#优化器
opt = tf.keras.optimizers.SGD(1e-3)
#迭代次数
num_epoch = 5
for e in range(num_epoch):
  loss = 0
  for i in range(len(F_train)):
    #使用tf.GradientTape()记录损失函数的梯度信息
    with tf.GradientTape() as tape:
        #预测值
        y = tf.matmul(tf.cast(b ,tf.float64),tf.expand_dims(F_train[i,:],axis=0),transpose_b=True) + tf.matmul(tf.cast(BETA ,tf.float64),tf.expand_dims(U_S_train[i,:],axis=0),transpose_b=True) + tf.cast(a ,tf.float64) # w * x_data + b
        #损失值
        loss += tf.reduce_mean(tf.square(y - Y_train[i]), name='loss') + tf.cast(tf.reduce_sum(tf.square(BETA)),tf.double)
  #自动计算loss 关于更新变量的梯度
  grads = tape.gradient(loss, variables)
  #自动根据梯度更新参数
  opt.apply_gradients(zip(grads, variables))
  if(e % 5 == 0):
      print(variables)


[<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.00172101], dtype=float32)>, <tf.Variable 'b:0' shape=(1, 10) dtype=float32, numpy=
array([[-0.74741143,  0.30653384,  0.4826346 ,  0.4595325 ,  0.505479  ,
         0.09858276,  0.89445263, -0.41733867,  0.9795727 , -0.539475  ]],
      dtype=float32)>, <tf.Variable 'BETA:0' shape=(1, 1000) dtype=float32, numpy=
array([[-8.22469950e-01, -7.76691496e-01,  9.72087502e-01,
         8.47213030e-01,  3.42215896e-01,  9.97877419e-02,
         4.59866881e-01,  5.43040037e-01,  5.67695022e-01,
         6.92423284e-01,  4.41049427e-01,  3.64338942e-02,
         9.23896194e-01,  3.94744486e-01,  8.09736788e-01,
         8.21765736e-02,  2.95991730e-02,  1.77687496e-01,
        -1.05224192e-01, -2.06984296e-01, -7.73559868e-01,
         1.08647496e-01,  8.74860644e-01,  5.17596304e-01,
         3.71069461e-01,  8.59199405e-01, -6.11765862e-01,
         4.71325517e-01, -1.49242640e-01, -9.60554063e-01,
        -7.07175970e-01,  3.

In [ ]:
# 模型测试
MSE = 0
Y_pred = []
for i in range(len(F_test)):
    y = tf.matmul(tf.cast(b ,tf.float64),tf.expand_dims(F_test[i,:],axis=0),transpose_b=True) + tf.matmul(tf.cast(BETA ,tf.float64),tf.expand_dims(U_S_test[i,:],axis=0),transpose_b=True) + tf.cast(a ,tf.float64) # w * x_data + b
    Y_pred.append(y)
    #损失值
    MSE += tf.reduce_mean(tf.square(y - Y_test[i]))
print("test MSE: ",MSE/len(Y_test))


test MSE:  tf.Tensor(96.14467276181844, shape=(), dtype=float64)


In [ ]:
list(Y_test)[:10]

[10.011507479861928,
 -2.888237756383442,
 3.379721669980119,
 10.006082725060818,
 -3.217158176943686,
 -0.3277494537509124,
 0.8386778490380048,
 -3.1096563011456735,
 4.137931034482775,
 0.0]

In [ ]:
Y_pred[:10]

[<tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[0.79703477]])>,
 <tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[-3.09522977]])>,
 <tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[2.73045211]])>,
 <tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[4.65604435]])>,
 <tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[0.19336824]])>,
 <tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[-0.41055805]])>,
 <tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[8.00904156]])>,
 <tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[-16.3073078]])>,
 <tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[2.31241193]])>,
 <tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[-1.82108644]])>]

In [ ]:
np.append(F_train,U_S_train)

array([ 0.48429433, -0.32117063, -0.40476755, ..., -0.01369483,
       -0.08253544, -0.04825744])

In [ ]:
# 模型训练
# lasso实现：
# lasso原理：https://zhuanlan.zhihu.com/p/116869931
from sklearn.linear_model import Lasso
model_lasso = Lasso()
model_lasso = model_lasso.fit(np.append(F_train,U_S_train).reshape((-1,k+S_word_num)), Y_train.reshape((-1,1))) #np.real(F)：提取复数的实部
'''
# get result: y = b0 + b1x
r_sq = model_Y_f.score(F, Y)
print('coefficient of determination(𝑅²) :', r_sq)
# coefficient of determination(𝑅²) : 0.715875613747954
print('intercept:', model_Y_f.intercept_)
# （标量） 系数b0 intercept: 5.633333333333329 -------this will be an array when y is also 2-dimensional
print('slope:', model_Y_f.coef_)
# （数组）斜率b1 slope: [0.54]        ---------this will be 2-d array when y is also 2-dimensional
'''
# predict response: given x, get y from the model y = b0+b1x
Y_lasso_pred = model_lasso.predict(np.append(F_test,U_S_test).reshape((-1,k+S_word_num)))

In [ ]:
Y_lasso_pred.shape

(589,)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_test,Y_lasso_pred)  #虽然比tf2的版本小，但是所有预测值都是一摸一样。。。

25.631424046093972

In [ ]:
Y_lasso_pred

array([-0.19257754, -0.19257754, -0.19257754, -0.19257754, -0.19257754,
       -0.19257754, -0.19257754, -0.19257754, -0.19257754, -0.19257754,
       -0.19257754, -0.19257754, -0.19257754, -0.19257754, -0.19257754,
       -0.19257754, -0.19257754, -0.19257754, -0.19257754, -0.19257754,
       -0.19257754, -0.19257754, -0.19257754, -0.19257754, -0.19257754,
       -0.19257754, -0.19257754, -0.19257754, -0.19257754, -0.19257754,
       -0.19257754, -0.19257754, -0.19257754, -0.19257754, -0.19257754,
       -0.19257754, -0.19257754, -0.19257754, -0.19257754, -0.19257754,
       -0.19257754, -0.19257754, -0.19257754, -0.19257754, -0.19257754,
       -0.19257754, -0.19257754, -0.19257754, -0.19257754, -0.19257754,
       -0.19257754, -0.19257754, -0.19257754, -0.19257754, -0.19257754,
       -0.19257754, -0.19257754, -0.19257754, -0.19257754, -0.19257754,
       -0.19257754, -0.19257754, -0.19257754, -0.19257754, -0.19257754,
       -0.19257754, -0.19257754, -0.19257754, -0.19257754, -0.19